# Numpy PoC

## Init

In [81]:
import copy
import numpy as np

rng = np.random.default_rng(seed=0)

def get_random_params(input_size, output_size, dtype=np.float32):
    W = rng.random(size=(output_size, input_size), dtype=dtype)
    b = rng.random(size=(output_size,), dtype=dtype)
    return W, b

def relu(x):
    return np.maximum(0, x)

class NN:
    def __init__(self, input_size=2, hidden_sizes=[3], output_size=1, activation=relu):
        self.input_size = input_size
        self.hidden_sizes = hidden_sizes
        self.output_size = output_size

        self.activation = activation
        # self.activation_func = relu

        params = [get_random_params(input_size, hidden_sizes[0])]
        for i in range(1, len(hidden_sizes)):
            params.append(get_random_params(hidden_sizes[i-1], hidden_sizes[i]))
        params.append(get_random_params(hidden_sizes[-1], output_size))

        self.Ws, self.Bs = zip(*params)
        self.Ws = list(self.Ws)
        self.Bs = list(self.Bs)

    def compute(self, x):
        for W, b in zip(self.Ws, self.Bs):
            x = self.activation(np.dot(W, x) + b)
        return x

    def permute(self, indices=[[0,1,2]]):
        permuted = copy.deepcopy(self)

        for i, perm_indices in enumerate(indices):
            permuted.Ws[i] = permuted.Ws[i][perm_indices,...]
            permuted.Bs[i] = permuted.Bs[i][perm_indices,...]
            permuted.Ws[i+1] = permuted.Ws[i+1][:,perm_indices]

        return permuted

    def __repr__(self):
        ret = f"NN(input_size={self.input_size}, hidden_sizes={self.hidden_sizes}, output_size={self.output_size})"
        ret += "\n"
        for i, (W, b) in enumerate(zip(self.Ws, self.Bs)):
            ret += f"Layer {i}:\nW_{i} {W.shape}:\n{W}\nb_{i} {b.shape}:\n{b}\n"
        return ret


## Define Net Params & Init Net

In [83]:
input_size = 3
hidden_sizes = [3, 3]
output_size = 3

net = NN(input_size=input_size, hidden_sizes=hidden_sizes, output_size=output_size)

## Forward Pass

In [84]:
x = rng.random(size=(input_size,))
y = net.compute(x)
y

array([6.63380037, 4.14094132, 6.35292454])

## Change Order of Neurons within Each Layer

In [90]:
net_permuted = net.permute(indices=[[1,2,0], [2,1,0]])

In [91]:
y_permuted = net_permuted.compute(x)
y_permuted

array([6.63380037, 4.14094132, 6.35292454])

### Assert Equality

In [87]:
np.allclose(y, y_permuted)

True

## Inspect Nets

In [88]:
net

NN(input_size=3, hidden_sizes=[3, 3], output_size=3)
Layer 0:
W_0 (3, 3):
[[0.08039963 0.29971188 0.48106134]
 [0.42268717 0.40323848 0.02831966]
 [0.00535262 0.12428325 0.00828427]]
b_0 (3,):
[0.6706244 0.5256177 0.6471895]
Layer 1:
W_1 (3, 3):
[[0.25729978 0.61538506 0.7640549 ]
 [0.38367754 0.4609216  0.9972099 ]
 [0.80498916 0.9808353  0.37952334]]
b_1 (3,):
[0.6855419  0.9501003  0.65045923]
Layer 2:
W_2 (3, 3):
[[0.84031135 0.6884467  0.704001  ]
 [0.38892138 0.8751561  0.13509649]
 [0.57890344 0.7214883  0.84548056]]
b_2 (3,):
[0.52535427 0.37541664 0.31024182]

In [89]:
net_permuted

NN(input_size=3, hidden_sizes=[3, 3], output_size=3)
Layer 0:
W_0 (3, 3):
[[0.42268717 0.40323848 0.02831966]
 [0.00535262 0.12428325 0.00828427]
 [0.08039963 0.29971188 0.48106134]]
b_0 (3,):
[0.5256177 0.6471895 0.6706244]
Layer 1:
W_1 (3, 3):
[[0.61538506 0.7640549  0.25729978]
 [0.4609216  0.9972099  0.38367754]
 [0.9808353  0.37952334 0.80498916]]
b_1 (3,):
[0.6855419  0.9501003  0.65045923]
Layer 2:
W_2 (3, 3):
[[0.84031135 0.6884467  0.704001  ]
 [0.38892138 0.8751561  0.13509649]
 [0.57890344 0.7214883  0.84548056]]
b_2 (3,):
[0.52535427 0.37541664 0.31024182]